# Estimating Transmissivity Using Specific Capacity Data

This notebook have the objective to estimate transmissivity from the Caçapava - SP well data. This will achieved by relating the available Specific Capacity data to transmissivity. After that the results of transmissivity estimates will be compared with the results from Iritani, 1999.

---

According to Mace [2001] three approach can be used to estimate transmissivity from specific capacity data:
* Analytical techniques
* Empirical techniques
* Geostatistical techniques

Analytical approach is not possible to use due the lack of storativity estimates/values. The estimates from Iritani Thesis is not feasible (Iritani, 1999). According to Mace [2001] the use of empirical techniques are <span style="color:red">(mais confiável para arbonates aquifers)</span> and for clastic aquifers the analytical approach is a better choice.

Analytical approach one should to coorect for partial-penetration effects and turbulent well loss. If the pumping rate is less than 600 gpm (136.27 $m^3 / h$) then the well losses due turbulent flow can be ignored.

PASTE For unconfined aquifers, correcting for
decreased saturated thickness might be necessary
(equation 76).

PASTE If the purpose of the data is to develop an interpolated map of transmissivity, then the geostatistical
approach should be considered. The choice of geostatistical approach, kriging with linear regression or
cokriging, depends on the number of transmissivity and
specific-eapacity pairs. If there are more than 50 transmissivity and specific-capacity pairs, then cokriging may
offer better results. If there are fewer than 50 pairs, then
kriging with linear regression might be better.

PASTE Before a decision to use the geostatistical approach
is made, the first step is to determine whether there are
enough transmissivity and specific capacity values for
the analysis. Generally the limitation will be the number
of transmissivity values because these values are rarer
than specific-capacity values. For kriging with linear
regression, there need to be 
* (1) enough transmissivity and
specific-capacity pairs to define a linear relationship
between transmissivity and specific capacity (similar to
the empirical approach) with a high correlation coefficient and uncorrelated residuals and 
* (2) enough
specific-capacity values and measured values of
transmissivity to define a semivariogram for each.

## Specific Capacity

Specific capacity is defined as well production per unit decline in head:

$S_c = \frac{Q}{s_w}$

where $Q$ is the pumping rate [$L^3$ e $t^-1$] and $S_w$ is the
measured drawdown (change in hydraulic head) in the well [$L$]. Note that the specific capacity has the same units as transmissivity.

Specific capacity is a function of:
* aquifer setting
  - transmissivity 
  - storativity
  - aquifer type
* well setting
  - well radius
  - degree of penetration
  - well losses due to well construction 
* pumping duration

## Empirical Method
PASTE Razack and Huntley (1991) found that the analytical
relationships (Thomasson and others, 1960; Theis and
others, 1963) uncorrected for well loss did not always
agree with measured values of transmissivity and that
empirical relationships produced less error than did
analytical solutions. The relationship is generally defined
between log-transformed values of transmissivity and
log-transformed values of specific capacity because
transmissivity and specific capacity are generally
lognormally distributed and, in this case, using log-log
relationships results in correlation coefficients that are better than those resulting from using untransformed
values (see, for example, Razack and Huntley, 1991;
EI-Naqa, 1994).

PASTE
The empirical approach involves 
* (1) compiling
all available aquifer-test information for an aquifer,
* (2) determining the transmissivity and specific capacity
for each of the tests, 
* (3) using regression to fit a line to
the plotted pairs of log transmissivity and log specific
capacity, and 
* (4) calculating the uncertainty in the linear
relationship.

<span style="color:red">(1) como nao achamos dados de teste de bombeamento iremos usar os resutlados de Iritani que foram obtidos atraves de dados requisitados na SABESP e em empresas privadas . Desta forma os passos (1) e (2) estão concluídos. Apesar de que os resultados de Iritani apresenta o passo (3) este sera refeito neste workflow como exercicio. </span>